# Big Data - Project

## TODO TOPIC

### Dragan Postolovsky
### Wojciech Taisner

TODO

A 1-page summary of litterature review on studies investigating stackoverflow data

TODO

An enumeration of "interesting" investigations directions for the 10-year dataset of stack overflow (from 09/2008 to 09/2018)

### What were the questions?

TODO

A report with analysis results (including visualisation, and statistics) of the 10 years data. These results are the fruit of the investigations projected earlier

In [1]:
!ls -la *.tar.xz

-rwxrwxrwx. 1 wojtek wojtek   204796672 01-09 21:50 Badges.tar.xz
-rw-rw-r--. 1 wojtek wojtek     2075408 01-15 21:50 BountyFrame-0.csv.tar.xz
-rwxrwxrwx. 1 wojtek wojtek  4093557860 01-10 13:47 Comments.tar.xz
-rw-rw-r--. 1 wojtek wojtek     8444828 01-15 19:56 DuplicateFrame-0.csv.tar.xz
-rwxrwxrwx. 1 wojtek wojtek    64935436 01-09 20:51 PostLinks.tar.xz
-rwxrwxrwx. 1 wojtek wojtek 12303248204 01-10 21:26 Posts.tar.xz
-rw-rw-r--. 1 wojtek wojtek      430164 01-15 18:44 TagsFrame-0.csv.tar.xz
-rwxrwxrwx. 1 wojtek wojtek      735732 01-09 21:32 Tags.tar.xz
-rw-rw-r--. 1 wojtek wojtek   179974748 01-15 21:25 UsersFrame-0.csv.tar.xz
-rwxrwxrwx. 1 wojtek wojtek   442348100 01-09 21:54 Users.tar.xz
-rwxrwxrwx. 1 wojtek wojtek  1002463504 01-09 21:28 Votes.tar.xz


In [2]:
# import dask bag
import dask.bag as db
# import dask datarfame
import dask.dataframe as dd
# import BeatuifulSoup
from bs4 import BeautifulSoup
# import pandas and numpy
import pandas as pd
import numpy as np

In [3]:
#files
base = "stackoverflow.com-"

badges = "Badges"
comments = "Comments"
posthistory = "PostHistory"
postlinks = "PostLinks"
posts = "Posts"
tags = "Tags"
users = "Users"
votes = "Votes"

all_files = [badges, comments, postlinks, posts, tags, users, votes] # posthistory, 
# for some reason post history does not want to co-operate
all_files

['Badges', 'Comments', 'PostLinks', 'Posts', 'Tags', 'Users', 'Votes']

# Functions

In [4]:
# non complex parsing (ex badges, tags, etc)
def xml_row_to_tuple(row): # gets data from single xml row (attr values)
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        if v.isdigit():
            v = int(v) #integers take less memory :D
        ret.append(v)
    return tuple(ret);


def xml_headers_to_tuple(row): # gets headers for single xml row (attr names)
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        ret.append(a)
    return tuple(ret);

In [5]:
# TODO
# read data only form provided attrs
def xml_row_to_tuple_attrs(row, d=tuple("Id")):
    soup = BeautifulSoup(row, "xml")
    ret = []
    for e in d:
        v = soup.row.attrs[e]
        if v != None:
            if v.isdigit():
                v=int(v)
            ret.append(v)
        else:
            ret.append(0)
    return tuple(ret)

In [6]:
## select shortest valid header
## if any field from tested header is missing in set of headers, then false
def is_header_valid(setof, element):
    for k in setof:
        for e in element:
            try:
                k.index(e)    
            except ValueError:
                return False
    return True


def find_valid_header(headers):
    # generate list of sorted headers by length ascending
    headers.sort(key=len)
    
    minlen = min(list(map(len, headers)))
    
    # for elements where len(element) == min(len(element)) - other just wont be valid anyway
    # try to find if it is a valid header
    for k in list(filter(lambda x: len(x) == minlen, headers)):
        if is_header_valid(headers, k):
            return k
    return tuple()


def shortest_valid_header(bag):
    # select distinct headers
    headers = bag.map(xml_headers_to_tuple).distinct().compute()
    return find_valid_header(headers)
    
    

In [7]:
# bag summary header + first row
# only bags with proper structure
def summary_bag(bag):
    th = bag.map(xml_headers_to_tuple).take(1)[0]
    for i, v in enumerate(th):
        print('{}: {}'.format(i, v))
    print("")
    tp = bag.map(xml_row_to_tuple).take(1)[0]
    for i, v in enumerate(tp):
        print('{}: {}'.format(i, v))
    print("")

In [8]:
# read bag function
def make_xml_bag(filename):
    extension=".tar.xz"
    compression="xz"
    temp_bag = db.read_text(filename+extension, compression=compression).str.strip()
    temp_bag = temp_bag.filter(lambda x: x.find("row") >= 0)
    return temp_bag

In [9]:
# check if bag is ok (data are more-a-less structured)
# takes a while of course
def check_bag(bag):
    ma = bag.map(lambda x: len(x)).max().compute()
    mi = bag.map(lambda x: len(x)).min().compute()
    if ma == mi:
        return True
    print(ma, mi)
    return False

In [10]:
# generate metadata iterable of tuples (name, dtype)
def meta_from_header(header):
    ret = []
    for k in header:
        if k.find("Date") >= 0:
            ret.append((k, np.dtype('datetime64[ns]')))
        elif k.find("Id") >= 0 or k.find("Score") >= 0 or k.find("Count") >= 0:
            ret.append((k, np.int_))
        else:
            ret.append((k, np.dtype('S'))) ## TODO some better parsing to string type
    return ret

In [ ]:
# TODO read, parse and store as csv
def full_parse(name, h=("Id")):
    return None

## Review of files structure (probably to be removed later)

In [69]:
# refiew structure of all files
# takes a second
#for filename in all_files:
#    print(filename)
#    temp_bag = make_xml_bag(filename)
#    summary_bag(temp_bag)

# Some real stuff

## Tags

In [15]:
# read tags bag, select rows only
tags_bag = db.read_text(tags+".tar.xz", compression="xz").str.strip()
tags_bag = tags_bag.filter(lambda x: x.find("row") >= 0)

### Simple tests [Training Field]

In [16]:
for row in tags_bag.take(4):
    print(row);
    print(xml_row_to_tuple(row))
    #soup = BeautifulSoup(row, "xml")
    #print(soup.row['Id'], soup.row['TagName'], soup.row['Count'])

<row Id="1" TagName=".net" Count="276857" ExcerptPostId="3624959" WikiPostId="3607476" />
(1, '.net', 276857, 3624959, 3607476)
<row Id="2" TagName="html" Count="789778" ExcerptPostId="3673183" WikiPostId="3673182" />
(2, 'html', 789778, 3673183, 3673182)
<row Id="3" TagName="javascript" Count="1723731" ExcerptPostId="3624960" WikiPostId="3607052" />
(3, 'javascript', 1723731, 3624960, 3607052)
<row Id="4" TagName="css" Count="563816" ExcerptPostId="3644670" WikiPostId="3644669" />
(4, 'css', 563816, 3644670, 3644669)


In [17]:
test = tags_bag.map(xml_row_to_tuple).take(1)
test[0]

(1, '.net', 276857, 3624959, 3607476)

In [18]:
header = tags_bag.map(xml_headers_to_tuple).take(1)[0]
header

('Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId')

In [19]:
print(type(tags_bag))
print(type(test))
#tags_bag.map(xml_row_to_tuple).take(1)

<class 'dask.bag.core.Bag'>
<class 'tuple'>


In [20]:
# takes a while
check_bag(tags_bag.map(xml_headers_to_tuple))

5 3


False

In [21]:
tags_bag.map(xml_row_to_tuple).filter(lambda x: len(x)<5).count().compute()

14218

In [22]:
tags_bag.map(xml_headers_to_tuple).filter(lambda x: len(x)==3).distinct().compute()
#tags_bag.count().compute()
#
# 14k / 55k has only 3/5 tags (two last missing), need to be dropped
#

[('Id', 'TagName', 'Count')]

In [23]:
%%time
dt = tags_bag.map(xml_headers_to_tuple).distinct().compute()
print(dt)

[('Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId'), ('Id', 'TagName', 'Count')]
CPU times: user 12.6 ms, sys: 42.3 ms, total: 55 ms
Wall time: 8.66 s


In [24]:
%%time
if len(dt) > 1:
    header = find_valid_header(dt)
    print(header)

('Id', 'TagName', 'Count')
CPU times: user 42 µs, sys: 24 µs, total: 66 µs
Wall time: 69.4 µs


In [25]:
%%time
tags_bag2 = tags_bag.map(xml_row_to_tuple_attrs, d=header)
print(check_bag(tags_bag2))

True
CPU times: user 41.3 ms, sys: 63.1 ms, total: 104 ms
Wall time: 21.6 s


In [26]:
#TODO export to dask dataframe and store
meta_from_header(header)
tags_df = tags_bag2.to_dataframe(meta=meta_from_header(header))
tags_df.to_csv("TagsFrame-*.csv.tar.xz",compression="xz",index=False)

['TagsFrame-0.csv.tar.xz']

In [27]:
tags_df.dtypes

Id          int64
TagName    object
Count       int64
dtype: object

In [28]:
tags_df.head()

,Id,TagName,Count
0,1,.net,276857
1,2,html,789778
2,3,javascript,1723731
3,4,css,563816
4,5,php,1245650


## Badges

In [11]:
badges_bag = make_xml_bag(badges)

In [65]:
%%time
## no need to run this, header provided next cell
#dba = badges_bag.map(xml_headers_to_tuple).distinct().compute()
#print(dba)

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 20 µs


In [12]:
%%time
## only one distinct header, do not need to compute this
dba = [('Id', 'UserId', 'Name', 'Date', 'Class', 'TagBased')]
if len(dba) > 1:
    b_header = find_valid_header(dba)
    print(b_header)

CPU times: user 19 µs, sys: 0 ns, total: 19 µs
Wall time: 44.8 µs


In [ ]:
%%time
#('Id', 'UserId', 'Name', 'Date', 'Class', 'TagBased')
temp_b = badges_bag.map(xml_row_to_tuple_attrs, d=dba[0]).to_dataframe(meta=meta_from_header(dba[0]))
temp_b.to_csv("BadgesFrame-*.csv.tar.xz",compression="xz",index=False)

## Comments

In [61]:
comments_bag = make_xml_bag(comments)

In [62]:
%%time
dcb = comments_bag.map(xml_headers_to_tuple).distinct().compute()
print(dcb)

[('Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'), ('Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserDisplayName', 'UserId'), ('Id', 'PostId', 'Score', 'Text', 'CreationDate'), ('Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserDisplayName')]
CPU times: user 6.35 s, sys: 2.32 s, total: 8.67 s
Wall time: 3h 47min 5s


In [63]:
%%time
## only one distinct header, do not need to compute this
if len(dcb) > 1:
    b_header = find_valid_header(dcb)
    print(b_header)
    
#('Id', 'PostId', 'Score', 'Text', 'CreationDate')

('Id', 'PostId', 'Score', 'Text', 'CreationDate')
CPU times: user 241 µs, sys: 0 ns, total: 241 µs
Wall time: 190 µs


## Post Links

In [29]:
postlinks_bag = make_xml_bag(postlinks)

In [30]:
%%time
dplb = postlinks_bag.map(xml_headers_to_tuple).distinct().compute()
print(dplb)

[('Id', 'CreationDate', 'PostId', 'RelatedPostId', 'LinkTypeId')]
CPU times: user 765 ms, sys: 275 ms, total: 1.04 s
Wall time: 14min 14s


In [31]:
%%time
## only one distinct header, do not need to compute this
if len(dplb) > 1:
    pl_header = find_valid_header(dplb)
    print(pl_header)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [51]:
%%time
##('Id', 'CreationDate', 'PostId', 'RelatedPostId', 'LinkTypeId')
temp_pl = postlinks_bag.filter(lambda x: x.find("LinkTypeId=\"3\"") >= 0).map(xml_row_to_tuple_attrs, d=dplb[0])
temp_pl = temp_pl.to_dataframe(meta=meta_from_header(dplb[0]))
temp_pl.to_csv("DuplicateFrame-*.csv.tar.xz",compression="xz",index=False)

CPU times: user 3min 33s, sys: 235 ms, total: 3min 34s
Wall time: 3min 34s


In [52]:
temp_pl.dtypes

Id                        int64
CreationDate     datetime64[ns]
PostId                    int64
RelatedPostId             int64
LinkTypeId                int64
dtype: object

In [46]:
temp_pl.head()

,Id,CreationDate,PostId,RelatedPostId,LinkTypeId
0,749670241,2013-02-18 03:03:16.917,70714,355934,3
1,749670246,2013-02-18 03:03:16.917,86947,73713,3
2,749670247,2013-02-18 03:03:16.917,97679,85122,3
3,749670248,2013-02-18 03:03:16.917,99408,1711,3
4,749670249,2013-02-18 03:03:16.917,26925,8127,3


In [50]:
temp_pl.count().compute()

Id               722449
CreationDate     722449
PostId           722449
RelatedPostId    722449
LinkTypeId       722449
dtype: int64

## Users

In [53]:
users_bag = make_xml_bag(users)

In [54]:
%%time
dub = users_bag.map(xml_headers_to_tuple).distinct().compute()
print(dub)

[('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'ProfileImageUrl', 'AccountId'), ('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'AccountId'), ('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'ProfileImageUrl', 'AccountId'), ('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Views', 'UpVotes', 'DownVotes', 'ProfileImageUrl', 'AccountId'), ('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'ProfileImageUrl', 'AccountId'), ('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Views', 'UpVotes', 'DownVotes', 'AccountId'), ('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', '

In [55]:
%%time
## only one distinct header, do not need to compute this
if len(dub) > 1:
    u_header = find_valid_header(dub)
    print(u_header)

('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'Views', 'UpVotes', 'DownVotes')
CPU times: user 104 µs, sys: 0 ns, total: 104 µs
Wall time: 109 µs


In [56]:
%%time
#('Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'Views', 'UpVotes', 'DownVotes')
temp_u = users_bag.map(xml_row_to_tuple_attrs, d=u_header)
temp_u.to_dataframe(meta=meta_from_header(u_header)).to_csv("UsersFrame-*.csv.tar.xz",compression="xz",index=False)

CPU times: user 57min 46s, sys: 6.97 s, total: 57min 53s
Wall time: 57min 52s


## Votes

In [59]:
votes_bag = make_xml_bag(votes)

In [49]:
%%time
### dont run this
### never ever
#CPU times: user 15 s, sys: 5.59 s, total: 20.6 s
#Wall time: 8h 16min 4sys
## result copied cell below

#dv = votes_bag.map(xml_headers_to_tuple).distinct().compute()
#print(dv)

[('Id', 'PostId', 'VoteTypeId', 'UserId', 'CreationDate'), ('Id', 'PostId', 'VoteTypeId', 'CreationDate', 'BountyAmount'), ('Id', 'PostId', 'VoteTypeId', 'UserId', 'CreationDate', 'BountyAmount'), ('Id', 'PostId', 'VoteTypeId', 'CreationDate')]
CPU times: user 15 s, sys: 5.59 s, total: 20.6 s
Wall time: 8h 16min 4s


In [61]:
%%time
dv = [('Id', 'PostId', 'VoteTypeId', 'UserId', 'CreationDate'), ('Id', 'PostId', 'VoteTypeId', 'CreationDate', 'BountyAmount'), ('Id', 'PostId', 'VoteTypeId', 'UserId', 'CreationDate', 'BountyAmount'), ('Id', 'PostId', 'VoteTypeId', 'CreationDate')]
## only one distinct header, do not need to compute this
if len(dv) > 1:
    v_header = find_valid_header(dv)
    print(v_header)

('Id', 'PostId', 'VoteTypeId', 'CreationDate')
CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 1.13 ms


In [63]:
%%time
temp_v = votes_bag.filter(lambda x: x.find("VoteTypeId=\"8\"") >= 0 or x.find("VoteTypeId=\"9\"") >=0)
temp_v = temp_v.map(xml_row_to_tuple_attrs, d=v_header)
temp_v.to_dataframe(meta=meta_from_header(v_header)).to_csv("BountyFrame-*.csv.tar.xz",compression="xz",index=False)

CPU times: user 7min 49s, sys: 707 ms, total: 7min 50s
Wall time: 7min 52s


## Posts History
seems that data are too big and too complicated to process them right now

## Posts and Answers 

In [15]:
## split posts to posts and answers
# takes a while
# quite a long ...
# man im gonna take a shower
%%time
both_bag = db.read_text(posts+".tar.xz", compression="xz").str.strip()
both_bag = both_bag.filter(lambda x: x.find('row') >=0 )
#totc = both_bag.count().compute()
#totc # total count

42850538

In [16]:
%%time
answers_bag = both_bag.filter(lambda x: x.find("PostTypeId=\"2\"") >= 0)
#answers_bag.map(xml_row_to_tuple).take(1)
#ansc = answers_bag.count().compute()
#ansc # answers count

25908201

In [17]:
%%time
posts_bag = both_bag.filter(lambda x: x.find("PostTypeId=\"1\"") >= 0)
#posts_bag.map(xml_row_to_tuple).take(1)
#posc = posts_bag.count().compute()
#posc # posts count

16845942

In [18]:
#missing = totc - ansc - posc # how many missing
#missing

96395

In [23]:
# try to find missing vol 2 works
#both_bag.filter(lambda x: x.find("PostTypeId=\"1\"") < 0 and x.find("PostTypeId=\"2\"") < 0).count().compute()

96395

In [68]:
# check whats wrong with data -> undefined types of posts
#both_bag.filter(lambda x: x.find("PostTypeId=\"1\"") < 0 and x.find("PostTypeId=\"2\"") < 0).take(1)

In [72]:
%%time
## check data structure across answers
## dont run it, takes a while
dab = answers_bag.map(xml_headers_to_tuple).distinct().compute()
#print(dab)

[('Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'Body', 'OwnerDisplayName', 'LastEditorUserId', 'LastEditDate', 'LastActivityDate', 'CommentCount'), ('Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'Body', 'OwnerUserId', 'LastEditorUserId', 'LastEditDate', 'LastActivityDate', 'CommentCount', 'CommunityOwnedDate'), ('Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'Body', 'OwnerUserId', 'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Tags', 'CommentCount', 'CommunityOwnedDate'), ('Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'Body', 'OwnerUserId', 'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'CommentCount'), ('Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'Body', 'OwnerDisplayName', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'CommentCount'), ('Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'Body', 'LastActivityDate', 'Comme

In [ ]:
%%time
## only one distinct header, do not need to compute this
if len(dab) > 1:
    ans_header = find_valid_header(dab)
    print(ans_header)

In [76]:
%%time
## check data structure across posts
## dont run it, takes a while
dpb = posts_bag.map(xml_headers_to_tuple).distinct().compute()
#print(dpb)

[('Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'LastEditorUserId', 'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'ClosedDate'), ('Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'ClosedDate'), ('Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerDisplayName', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount'), ('Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'ClosedDate'), ('Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerDisplayName', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',

In [ ]:
%%time
## only one distinct header, do not need to compute this
if len(dpb) > 1:
    pos_header = find_valid_header(dpb)
    print(pos_header)